# 06. CODE SUMMARIZATION

1. [x] введение
2. [x] данные
3. [x] BLEU
4. [x] упражение
5. [x] ссылки

# 1. Введение

[Обратная задача](https://proceedings.neurips.cc/paper/2019/file/e52ad5c9f751f599492b4f087ed7ecfc-Paper.pdf) к задаче генерации кода:
- синтез кода --- по описанию создать код;
- суммаризация --- по коду создать описание.

В целом, задача суммаризации (генерация docstring-а в случае Python) похожа на задачу генерации кода:
- можно использовать аналогичную архитектуру
- можно использовать аналогичные данные (вместо пары текст-код нужна пара код-текст)
- НО нужна другая функция потерь, не можем использовать тесты для фильтрации или проверки


Например, результаты использования модели генерации кода [InCode](https://arxiv.org/abs/2204.05999) для задачи суммаризации:

| method | BLEU|
| - | - |
| InCoder | 16.05--18.27 |
| RoBERTa (FT) | 18.14 |
| CodeBERT (FT) | 19.06 |
| PLBART (FT) | 19.30 |
| CodeT5 (FT) | 20.36 |

# 2. Данные

## Обучение

Обычно данные для code summarization или code generation --- это пары: код и описание кода на естественном языке.

- Java dataset: [Hu et al. 2018](https://xin-xia.github.io/publication/icpc182.pdf)
- Python dataset: [Wei et al. 2018](https://arxiv.org/abs/1910.05923)
- [CodeSearchNet](https://github.com/github/CodeSearchNet) [[Husain et al. 2019](https://github.com/github/CodeSearchNet)]: 2 миллиона пар (комментарий, код); языки: Python, Javascript, Ruby, Go, Java, PHP
- CoCoNet: [Wang et al. 2021](https://arxiv.org/abs/2107.01933): 1.2 миллиона пар, Java

Инструменты для сбора своего датасета:
- [code-summarization-dataset](https://github.com/JetBrains-Research/code-summarization-dataset)

## CodeXGLUE

CodeSerachNet входит в бенчмарк [CodeXGLUE](https://arxiv.org/abs/2102.04664).
CodeXGLUE (Microsoft) включает в себя [набор](https://github.com/microsoft/CodeXGLUE) задач анализа кода и платформу для оценки и сравнения моделей.
CodeXGLUE расшифровывается как *General Language Understanding Evaluation* для *CODE*.
Он включает в себя 14 наборов данных для 10 разнообразных задач анализа кода, охватывающих следующие сценарии:
- код-код (обнаружение клонов, обнаружение дефектов и т.д.)
- text-code (поиск в коде, генерация кода)
- код-текст (суммаризация)
- текст-текст (перевод документации)

![](https://raw.githubusercontent.com/microsoft/CodeXGLUE/main/tasks.jpg)

# 3.  BLEU

BLEU (bilingual evaluation understudy, [Papineni et al. 2002](https://aclanthology.org/P02-1040/)) --- алгоритм оценки качества текста, который был машинно переведен с одного естественного языка на другой.
Качество считается соответствием между работой машины и человека: "чем ближе машинный перевод к профессиональному человеческому переводу, тем он лучше".
BLEU была одной из первых метрик, заявивших о высокой корреляции с человеческими суждениями о качестве, и остается одной из самых популярных автоматических и недорогих метрик.

Баллы рассчитываются для отдельных переведенных сегментов (как правило, предложений) путем сравнения их с набором эталонных переводов хорошего качества. Эти оценки затем усредняются по всему корпусу, чтобы получить оценку общего качества перевода.

Результатов оценки BLEU всегда представляет собой число от 0 до 1. Это значение указывает, насколько текст-кандидат похож на справочные тексты (чем больше, тем лучше).

Как вычислить BLEU?

### Сначала Precision

1. для каждой $n$-граммы из предложения-кандидата считаем, сколько раз она встречается в исходном предложении
2. суммируем общее количество вхождений для каждой уникальной $n$-граммы из предложения-кандидата
3. и делим на количество $n$-грамм в предложении-кандидате.

*Пример 1*:
- для простоты, $n = 1$, униграмма --- отдельное слово
- исходное предложение: `the cat is on the mat`
- предложение-кандидат: `the cat the cat on the mat`
- все $n$-граммы предложения-кандидата: `the` `cat` `on` `mat`


|кандидат     | the | cat | the | cat | on | the | mat |
|-------------|-----|-----|-----|-----|----|-----|-----|
|встречается? | 1   | 1   | 1   | 1   | 1  | 1   | 1   |

Суммируем:

|             | the | cat | on | mat |
|-------------|-----|-----|----|-----|
|сколько раз? |  3  |  2  |  1 |  1  |

Таким образом, общее количество вхождения равно для каждой уникальной $n$-граммы из предложения-кандидата $3+2+1+1=7$. При этом, количестов $n$-грамм в предложении-кандидате тоже равно $7$. Таким образом, precision равно $\frac{7}{7} = 1$.


*Пример 2*:
- исходное предложение: `the cat is on the mat`
- предложение-кандидат: `the cat this cat on the mat`
- все $n$-граммы предложения-кандидата: `the` `cat` `this` `on` `mat`


|кандидат     | the | cat | this | cat | on | the | mat |
|-------------|-----|-----|------|-----|----|-----|-----|
|встречается? | 1   | 1   | 0    | 1   | 1  | 1   | 1   |

Суммируем:

|             | the | cat | this | on | mat |
|-------------|-----|-----|------|----|-----|
|сколько раз? |  2  |  2  |  0   |  1 |  1  |

Общее количество вхождения равно для каждой уникальной $n$-граммы из предложения-кандидата $2+2+0+1+1=6$. При этом, количестов $n$-грамм в предложении-кандидате тоже равно $7$. Таким образом, precision равно $\frac{6}{7}$.

### Потом Modified precision

1. для каждой $n$-граммы из предложения-кандидата определям, встречается ли она в исходном предложении
2. суммируем общее количество вхождений для каждой уникальной $n$-граммы из предложения-кандидата
3. считаем минимум между полученным значением и количеством вхождений в исходное предложение
4. и делим на количество $n$-грамм в предложении-кандидате.

*Пример*:
- исходное предложение: `the cat is on the mat`
- предложение-кандидат: `the cat the cat on the mat`


|кандидат     | the | cat | the | cat | on | the | mat |
|-------------|-----|-----|-----|-----|----|-----|-----|
|встречается? |True |True |True |True |True|True |True |

Суммируем:

|             | the | cat | on | mat |
|-------------|-----|-----|----|-----|
|сколько раз? |  3  |  2  |  1 |  1  |

Минимизируем:

|             | the | cat | on | mat |
|-------------|-----|-----|----|-----|
|сколько раз? |  2=min(3,2)  |  1=min(2,1)  |  1 |  1  |


Общее количество вхождений для уникальных $n$-грамм в исходном предложении равно $5=2+1+1+1$, и общее количество униграмм в предложении-кандидате равно $7$. Таким образом, modified precision равно $\frac{5}{7}$.

Теперь для каждой пары: (предложение-кандидат и исходное предложение) мы можем считать modified precision. Если исходных предложений несколько (например, несколько правильных переводов), то имеем свой modified precision (обозначается через $p_n)$ для каждой пары.

### Теперь [BLEU](https://en.wikipedia.org/wiki/BLEU)

$$BLEU = BP⋅\exp \left(\sum_{n=1}^N w_n \ln p_n \right)$$
где
- $N$ --- верхняя граница для $n$ (например, $\infty$)
- $p_n$ --- это modified precision для $n$-грамм,
- $w_n$ --- фиксированный набор весовых коэффициентов таких, что $\sum_{n=1}^N w_n = 1$, $w_n \geq 0$,
- $BP$ --- штраф за краткость (штрафуем короткие машинные переводы).

Пусть
- $c$ --- длина предложения-кандидата (или сумма длин, если их несколько),
а
- $r$ --- сумма длин наилучших совпадений каждого предложения-кандидата с исходными предложениями (если у нас одно предложение-кандидат, что имеем длину наиболее близкого по длине исходного предложения).

Тогда

$BP = \left\{
  \begin{array}{ c l }
    1 & \quad \textrm{если } c > r \\
    \exp(1−\frac{r}{c}) & \quad \textrm{иначе}
  \end{array}
\right.$


> Пример:
> Здесь наилучшей длиной совпадения является ближайшая к предложениям-кандидатам длина исходного предложения.
> Например, если есть три исходных предложения длины $12$, $14$ и $17$ слов,
> а предложение-кандидат содержит $13$ слов,
> то теоретически наилучшей длиной совпадения может быть либо $12$, либо $14$,
> но мы выбираем более короткую, которая равна $12$ ($r = 12$).

Обычно BLEU оценивается на корпусе, где есть много предложений-кандидатов, переведенных из разных исходных текстов, и каждое из них имеет исхожных предложений.


Нетрудно обнаружить, что BLEU всегда имеет значение от $0$ до $1$.
Действительно, $BP$, $w_n$ и $p_n$ всегда от $0$ до $1$, и
  
$$\exp \left(\sum_{n=1}^N w_n \ln p_n \right)=\prod_{n=1}^N \exp\left(w_n \ln p_n \right)=\prod_{n=1}^N\left[\exp(\ln p_n)\right]^{w_n}= \prod_{n=1}^N p_n^{w_n} \in [0, 1]$$

Обычно, для BLEU используют $N=4$ и $w_n=\frac{1}{N}$.


> Пример: Несложно вычислить modified precision для остальных $n$-грамм.
> Итак,
> 
> $$p_1=\frac{5}{7}, p_2=\frac{4}{6}, p_3=\frac{2}{5}, p_4=\frac{1}{4}$$
> $$w_1=w_2=w_3=w_4=\frac{1}{4}$$
> 
> Поскольку в корпусе имеется только один набор переводов, имеем $c=7$ и $r=7$.
> 
> $$BP=1$$
> 
> Мы подставляем эти значения к уравнению BLEU, и получаем
> 
> $$BLEU=0.467$$

In [ ]:
import nltk

references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]

score = nltk.translate.bleu_score.corpus_bleu(references, candidates, weights=(0.25,0.25,0.25,0.25))
print(score)

### METEOR

[METEOR](https://en.wikipedia.org/wiki/METEOR) (Metric for Evaluation of Translation with Explicit ORdering, [Banerjee, Lavie 2005](https://aclanthology.org/W05-0909/)) — метрика для оценки качества машинного перевода. Метрика базируется на использовании $n$-грамм и ориентирована на использование статистической и точной оценки исходного текста. В отличие от метрики BLEU, данная метрика использует функции сопоставления синонимов вместе с точным соответствием слов. Метрика была разработана, чтобы решить проблемы, которые были найдены в более популярной метрике BLEU, а также создать хорошую корреляцию с оценкой экспертов на уровне словосочетаний или предложений. 

### ROUGE-L

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) (Recall-Oriented Understudy for Gisting Evaluation, [Lin 2004](https://aclanthology.org/W04-1013/)) --- набор показателей и программный пакет, используемый для оценки программного обеспечения автоматического суммирования и машинного перевода при обработке естественного языка. Метрики сравнивают автоматически созданную сводку или перевод со справочной информацией или набором ссылок (созданным человеком) сводкой или переводом.

### CIDEr

[CIDEr](https://arxiv.org/abs/1411.5726) (Consensus-based Image Description Evaluation, [Vedantam et al. 2015](https://arxiv.org/abs/1411.5726))

###  CodeBLEU (не для суммаризации)

BLEU разработан для NLP и не учитывает отличия языков программирования от естественных языков.
- небольшое количество ключевых слов в языках программирования; при этом ключевые слова более важные, чем остальные
- код имее древовидную структуру
- код понимается однозначно

Поэтому был предложен [CodeBLEU](https://arxiv.org/abs/2009.10297) (Microsoft, 2020) --- использует $n$-граммы, AST и потоки данных (data-flow).

![](./res/06_codebleu_formula.png)

$$CodeBLEU = \alpha \cdot BLEU + \beta \cdot BLEU_{weight} + \gamma \cdot Match_{ast} + \delta \cdot Match_{df}$$

- $\alpha = \beta = \gamma = \delta = 0.25$
- $BLEU$ --- стандартный $BLEU$
- $BLEU_{weight}$ --- взвешанный $BLEU$, учитывает важность отдельных слов (ключевых слов): значение $p_n$ учитывает веса $n$-граммы. Например, в исходной работе $N = 1$, $w_1 = 1$, ключевые слова имеют вес в $5$ раз больше
- $Match_{ast}$ --- мера похожести AST-деревьев. Из деревьев выкидываются листья, поскольку в них хранятся имена функций, значения переменных. Сравнивается количество поддеревьев, построенных библиотекой `tree-sitter`. $Match_{ast} = \frac{Count_{clip}(T_{cand})}{Count(T_{ref})}$,
 где $Count(T_{ref})$ --- общее количество поддеревьев у референсного кода, $Count_{clip}(T_{cand})$ --- количество поддеревьев в коде-кандидате, которые соответствуют (в некотором смысле) референсному коду.
- $Match_{df}$ --- строится граф потока данных (аналогично [GraphCodeBERT](https://arxiv.org/abs/2009.08366)): узлы --- переменные, дуги графа --- откуда возникла каждая переменная. Оцениваем похожесть этих графов (отношение количества переменных). 

![](./res/06_codebleu_dataflow.png)


> Примеры
> ![](./res/06_codebleu_example_1.png)
> ![](./res/06_codebleu_example_2.png)

# 4. Упражнение

На основе существующих открытых обученных моделей (CodeBERT, InCoder) собрать решение для суммаризации кода.

# 5. Полезные ссылки

- [Code summarization](https://github.com/saltudelft/ml4se#code-summarization)
- [Recommendations for Datasets for Source Code Summarization](https://arxiv.org/abs/1904.02660)
- [Ahmad et al - A Transformer-based Approach for Source Code Summarization 2021](https://arxiv.org/abs/2005.00653)
- [Wang et al - CoCoSum: Contextual Code Summarization with Multi-Relational Graph Neural Network 2021 (Microsoft)](https://arxiv.org/abs/2107.01933)
- [Shi et al - On the Evaluation of Neural Code Summarization 2021 (Microsoft)](https://arxiv.org/abs/2107.07112)
- [Rauf et al - Meta Learning for Code Summarization 2022](https://arxiv.org/abs/2201.08310)